# imports & setup

In [1]:
import logging
import os
from pprint import pprint

import dask.dataframe as dd
import helpers
import numpy as np
import pandas as pd
import scipy.stats
from cloudpathlib import AnyPath as Path

In [2]:
import warnings

warnings.filterwarnings("ignore")

In [3]:
handler = logging.StreamHandler()
formatter = logging.Formatter(
    "%(asctime)s %(process)d/%(threadName)s %(name)s %(levelname)s\n%(message)s"
)
handler.setFormatter(formatter)
logging.getLogger().handlers = [handler]

In [4]:
logging.getLogger("gcsfs").setLevel("DEBUG")
logging.getLogger("google.cloud.bigquery").setLevel("DEBUG")
logging.getLogger("helpers").setLevel("DEBUG")
logging.getLogger("pandas").setLevel("DEBUG")
logging.getLogger("pyarrow").setLevel("DEBUG")

In [5]:
logger = logging.getLogger(__name__)
logger.setLevel("DEBUG")
logger.debug("test debug-level message")

2022-07-16 16:01:07,215 17392/MainThread __main__ DEBUG
test debug-level message


# loading data

## TCGA SKCM (real) bulk rna-seq

In [6]:
uri_tcga_skcm_bulk_rnaseq = (
    "gs://liulab/data/pseudobulk_optimization/3_with_tcga_qc/"
    "mixtures_real_tcga_skcm/tpm.parquet"
)

In [ ]:
!gsutil ls -lh $uri_tcga_skcm_bulk_rnaseq

In [7]:
ddf_tcga_skcm_bulk_rnaseq = dd.read_parquet(uri_tcga_skcm_bulk_rnaseq, engine="pyarrow")
logger.debug(ddf_tcga_skcm_bulk_rnaseq.dtypes)

2022-07-16 16:01:20,755 17392/MainThread gcsfs.credentials DEBUG
Connected with method google_default
2022-07-16 16:01:20,757 17392/fsspecIO gcsfs DEBUG
GET: b/{}/o/{}, ('liulab', 'data/pseudobulk_optimization/3_with_tcga_qc/mixtures_real_tcga_skcm/tpm.parquet'), None
2022-07-16 16:01:20,758 17392/fsspecIO gcsfs.credentials DEBUG
GCS refresh
2022-07-16 16:01:20,915 17392/fsspecIO gcsfs DEBUG
GET: b/{}/o/{}, ('liulab', 'data/pseudobulk_optimization/3_with_tcga_qc/mixtures_real_tcga_skcm/tpm.parquet'), None
2022-07-16 16:01:20,946 17392/fsspecIO gcsfs DEBUG
GET: b/{}/o/{}, ('liulab', 'data/pseudobulk_optimization/3_with_tcga_qc/mixtures_real_tcga_skcm/tpm.parquet'), None
2022-07-16 16:01:20,977 17392/fsspecIO gcsfs DEBUG
GET: https://storage.googleapis.com/download/storage/v1/b/liulab/o/data%2Fpseudobulk_optimization%2F3_with_tcga_qc%2Fmixtures_real_tcga_skcm%2Ftpm.parquet?alt=media, (), {'Range': 'bytes=47361643-47427178'}
2022-07-16 16:01:21,079 17392/fsspecIO gcsfs DEBUG
GET: b/{}/o/{

In [19]:
ddf_tcga_skcm_bulk_rnaseq

,gene_symbol,aliquot_barcode,tpm
npartitions=1,,,
,category[unknown],category[unknown],float64
,...,...,...


## TCGA SKCM fractions (estimated)

In [16]:
df_tcga_skcm_fractions_from_csx = helpers.datasets.load_tcga_skcm_fractions_from_csx()

2022-07-16 16:18:16,288 17392/MainThread helpers.datasets DEBUG
loading TCGA SKCM fractions estimated by CIBERSORTx
2022-07-16 16:18:16,290 17392/fsspecIO gcsfs DEBUG
GET: b/{}/o/{}, ('liulab', 'downloaded_manually/derek_csx_tcga_skcm/CIBERSORTx_Job8_Results.txt'), None
2022-07-16 16:18:16,337 17392/fsspecIO gcsfs DEBUG
GET: https://storage.googleapis.com/download/storage/v1/b/liulab/o/downloaded_manually%2Fderek_csx_tcga_skcm%2FCIBERSORTx_Job8_Results.txt?alt=media, (), {'Range': 'bytes=0-86860'}


In [17]:
immune_cell_types = ["B", "Macrophage", "NK", "T", "T CD4", "T CD8"]

df_immune_fraction_by_aliquot_barcode = (
    df_tcga_skcm_fractions_from_csx[immune_cell_types]
    .sum(axis="columns")
    .rename("immune_fraction")
    .to_frame()
)

In [18]:
df_immune_fraction_by_aliquot_barcode.sample(5)

,immune_fraction
sample_id,
TCGA-D3-A2J6-06A-11R-A18T-07,0.041182
TCGA-D9-A1X3-06A-11R-A18S-07,0.005262
TCGA-FW-A3R5-06A-11R-A239-07,0.056826
TCGA-ER-A199-06A-11R-A18T-07,0.197912
TCGA-ER-A19L-06A-12R-A18S-07,0.019266


In [ ]:
ddf_bulk_rnaseq_all_with_immune = ddf_bulk_rnaseq_all.merge(
    df_immune_fraction_by_aliquot_barcode,
    left_on="aliquot_barcode",
    right_on="sample_id",
)

ddf_bulk_rnaseq_all_with_immune

## pseudo bulk rna-seq

In [8]:
uri_pseudobulk_rnaseq = (
    "gs://liulab/data/pseudobulk_optimization/3_with_tcga_qc/"
    "mixtures/n_cells=5/malignant_from_one_sample=True/data.parquet"
)

In [ ]:
!gsutil ls -lh $uri_pseudobulk_rnaseq

In [9]:
ddf_pseudobulk_rnaseq = (
    dd.read_parquet(
        uri_pseudobulk_rnaseq,
        engine="pyarrow",
    )
    .replace({"malignant_from_one_sample": {"True": True, "False": False}})
    .astype({"n_cells": "uint8", "malignant_from_one_sample": "bool"})
    .rename(columns={"tcga_aliquot_barcode_for_fractions": "aliquot_barcode"})
)

logger.debug(ddf_pseudobulk_rnaseq.dtypes)

2022-07-16 16:02:00,238 17392/fsspecIO gcsfs DEBUG
GET: b/{}/o/{}, ('liulab', 'data/pseudobulk_optimization/3_with_tcga_qc/mixtures/n_cells=5/malignant_from_one_sample=True/data.parquet'), None
2022-07-16 16:02:00,277 17392/fsspecIO gcsfs DEBUG
GET: b/{}/o/{}, ('liulab', 'data/pseudobulk_optimization/3_with_tcga_qc/mixtures/n_cells=5/malignant_from_one_sample=True/data.parquet'), None
2022-07-16 16:02:00,311 17392/fsspecIO gcsfs DEBUG
GET: b/{}/o/{}, ('liulab', 'data/pseudobulk_optimization/3_with_tcga_qc/mixtures/n_cells=5/malignant_from_one_sample=True/data.parquet'), None
2022-07-16 16:02:00,339 17392/fsspecIO gcsfs DEBUG
GET: https://storage.googleapis.com/download/storage/v1/b/liulab/o/data%2Fpseudobulk_optimization%2F3_with_tcga_qc%2Fmixtures%2Fn_cells=5%2Fmalignant_from_one_sample=True%2Fdata.parquet?alt=media, (), {'Range': 'bytes=41829238-41894773'}
2022-07-16 16:02:00,418 17392/fsspecIO gcsfs DEBUG
GET: b/{}/o/{}, ('liulab', 'data/pseudobulk_optimization/3_with_tcga_qc/mixtur

In [20]:
ddf_pseudobulk_rnaseq

,gene_symbol,aliquot_barcode,tpm,n_cells,malignant_from_one_sample
npartitions=1,,,,,
,category[unknown],category[unknown],float64,uint8,bool
,...,...,...,...,...


In [14]:
df_pseudobulk_rnaseq = (
    dd.read_parquet(
        uri_pseudobulk_rnaseq,
        engine="pyarrow",
    )
    .replace({"malignant_from_one_sample": {"True": True, "False": False}})
    .astype({"n_cells": "uint8", "malignant_from_one_sample": "bool"})
    .rename(columns={"tcga_aliquot_barcode_for_fractions": "aliquot_barcode"})
).compute()

logger.debug(df_pseudobulk_rnaseq.dtypes)
df_pseudobulk_rnaseq.info()

2022-07-16 16:12:27,315 17392/fsspecIO gcsfs DEBUG
GET: b/{}/o/{}, ('liulab', 'data/pseudobulk_optimization/3_with_tcga_qc/mixtures/n_cells=5/malignant_from_one_sample=True/data.parquet'), None
2022-07-16 16:12:27,356 17392/fsspecIO gcsfs DEBUG
GET: b/{}/o/{}, ('liulab', 'data/pseudobulk_optimization/3_with_tcga_qc/mixtures/n_cells=5/malignant_from_one_sample=True/data.parquet'), None
2022-07-16 16:12:27,383 17392/fsspecIO gcsfs DEBUG
GET: b/{}/o/{}, ('liulab', 'data/pseudobulk_optimization/3_with_tcga_qc/mixtures/n_cells=5/malignant_from_one_sample=True/data.parquet'), None
2022-07-16 16:12:27,415 17392/fsspecIO gcsfs DEBUG
GET: https://storage.googleapis.com/download/storage/v1/b/liulab/o/data%2Fpseudobulk_optimization%2F3_with_tcga_qc%2Fmixtures%2Fn_cells=5%2Fmalignant_from_one_sample=True%2Fdata.parquet?alt=media, (), {'Range': 'bytes=41829238-41894773'}
2022-07-16 16:12:27,494 17392/fsspecIO gcsfs DEBUG
GET: b/{}/o/{}, ('liulab', 'data/pseudobulk_optimization/3_with_tcga_qc/mixtur

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5911184 entries, 0 to 5911183
Data columns (total 5 columns):
 #   Column                     Dtype   
---  ------                     -----   
 0   gene_symbol                category
 1   aliquot_barcode            category
 2   tpm                        float64 
 3   n_cells                    uint8   
 4   malignant_from_one_sample  bool    
dtypes: bool(1), category(2), float64(1), uint8(1)
memory usage: 79.6 MB


## determine immune high and low

In [ ]:
ddf_tcga_skcm_bulk_rnaseq.join(df_immune_fraction_by_aliquot_barcode)

# analysis

### analysis - computing stats on each dataset individually

In [ ]:
ddf_tcga_skcm_bulk_rnaseq.join(df_immune_fraction_by_aliquot_barcode)

immune_threshold_low = df_rnaseq_with_immune["immune_fraction"].quantile(0.2)
immune_threshold_high = df_rnaseq_with_immune["immune_fraction"].quantile(0.8)

### analysis - combining all data, then computing stats

In [ ]:
# merge real and pseudo data
ddf_bulk_rnaseq_all = ddf_tcga_skcm_bulk_rnaseq.merge(
    ddf_pseudobulk_rnaseq,
    on=["aliquot_barcode", "gene_symbol"],
    how="inner",
    suffixes=["_tcga_skcm", "_pseudo"],
)

In [ ]:
ddf_bulk_rnaseq_all

### join these fractions onto bulk rna-seq data


In [ ]:
ddf_bulk_rnaseq_all_with_immune = ddf_bulk_rnaseq_all.merge(
    df_immune_fraction_by_aliquot_barcode,
    left_on="aliquot_barcode",
    right_on="sample_id",
)

ddf_bulk_rnaseq_all_with_immune

In [ ]:
# compute immune high and low
immune_threshold_low = ddf_bulk_rnaseq_all_with_immune["immune_fraction"].quantile(0.2)
immune_threshold_high = ddf_bulk_rnaseq_all_with_immune["immune_fraction"].quantile(0.8)

In [ ]:
immune_threshold_low, immune_threshold_high

In [ ]:
ddf_bulk_rnaseq_all_with_immune["immune_low"] = (
    ddf_bulk_rnaseq_all_with_immune["immune_fraction"] <= immune_threshold_low
)

ddf_bulk_rnaseq_all_with_immune["immune_high"] = (
    ddf_bulk_rnaseq_all_with_immune["immune_fraction"] >= immune_threshold_high
)

In [ ]:
df_bulk_rnaseq_all_with_immune = ddf_bulk_rnaseq_all_with_immune.compute()

In [ ]:
df_bulk_rnaseq_all_with_immune["aliquot_barcode"].value_counts()

In [ ]:
df_bulk_rnaseq_all_with_immune[["immune_low", "immune_high"]].value_counts()

#### compute stats for each gene

In [ ]:
### compute p-values...


def compute_stats(df):
    immune_low = df[df["immune_low"]]
    immune_high = df[df["immune_high"]]
    pval_pseudo = scipy.stats.mannwhitneyu(
        immune_high["tpm_pseudo"].values, immune_low["tpm_pseudo"].values
    )[1]
    neglog10pval_pseudo = -np.log10(pval_pseudo)
    foldchange_pseudo = (
        immune_high["tpm_pseudo"].mean() / immune_low["tpm_pseudo"].mean()
    )
    log2foldchange_pseudo = np.log2(foldchange_pseudo)

    pval_real = scipy.stats.mannwhitneyu(
        immune_high["tpm_tcga_skcm"].values, immune_low["tpm_tcga_skcm"].values
    )[1]
    neglog10pval_real = -np.log10(pval_real)
    foldchange_real = (
        immune_high["tpm_tcga_skcm"].mean() / immune_low["tpm_tcga_skcm"].mean()
    )
    log2foldchange_real = np.log2(foldchange_real)

    return pd.Series(
        dict(
            pval_pseudo=pval_pseudo,
            foldchange_pseudo=foldchange_pseudo,
            log2foldchange_pseudo=log2foldchange_pseudo,
            neglog10pval_pseudo=neglog10pval_pseudo,
            signedneglog10pval_pseudo=(
                neglog10pval_pseudo * np.sign(log2foldchange_pseudo)
            ),
            pval_real=pval_real,
            foldchange_real=foldchange_real,
            log2foldchange_real=log2foldchange_real,
            neglog10pval_real=neglog10pval_real,
            signedneglog10pval_real=(neglog10pval_real * np.sign(log2foldchange_real)),
        )
    )


df_gene_stats_by_immune = df_bulk_rnaseq_all_with_immune.groupby("gene_symbol").apply(
    compute_stats
)
df_gene_stats_by_immune = df_gene_stats_by_immune.reset_index()

In [ ]:
df_gene_stats_by_immune

In [ ]:
# how many genes have valid stats?

logger.debug(df_gene_stats_by_immune["pval_pseudo"].isna().value_counts())
logger.debug(df_gene_stats_by_immune["foldchange_pseudo"].isna().value_counts())
logger.debug(df_gene_stats_by_immune["pval_real"].isna().value_counts())
logger.debug(df_gene_stats_by_immune["foldchange_real"].isna().value_counts())

In [ ]:
import plotly.express as px

In [ ]:
fig = px.scatter(
    df_gene_stats_by_immune,
    x="log2foldchange_real",
    y="neglog10pval_real",
    title="real (tcga skcm): immune high / low",
    hover_name="gene_symbol",
    hover_data=["foldchange_real", "pval_real"],
)
fig.update_xaxes(range=(-10, 10))
fig.update_yaxes(range=(0, 30))
fig.update_traces(marker=dict(size=3))
fig.show(renderer="png", scale=1, width=1000, height=500)

In [ ]:
fig.show(renderer="browser")

In [ ]:
fig = px.scatter(
    df_gene_stats_by_immune,
    x="log2foldchange_pseudo",
    y="neglog10pval_pseudo",
    title="pseudobulks: immune high / low",
    hover_name="gene_symbol",
    hover_data=["foldchange_pseudo", "pval_pseudo"],
)
fig.update_xaxes(range=(-10, 10))
fig.update_yaxes(range=(0, 30))
fig.update_traces(marker=dict(size=3))
fig.show(renderer="png", scale=2, width=800, height=600)

In [ ]:
fig.show(renderer="browser")

In [ ]:
fig = px.scatter(
    df_gene_stats_by_immune,
    x="signedneglog10pval_real",
    y="signedneglog10pval_pseudo",
    trendline="ols",
    title="signed -log10(p-values): pseudobulks vs tcga skcm",
    hover_name="gene_symbol",
    hover_data=["log2foldchange_real", "log2foldchange_pseudo"],
)
fig.update_xaxes(range=(-25, 25))
fig.update_yaxes(range=(-25, 25))
fig.update_traces(marker=dict(size=3))
fig.show(renderer="png", scale=1, width=500, height=500)

In [ ]:
fig.show(renderer="browser")

In [ ]:
df_gene_stats_by_immune.info()

In [ ]:
fig = px.scatter(
    df_gene_stats_by_immune,
    x="log2foldchange_real",
    y="log2foldchange_pseudo",
    trendline="ols",
    title="logs(fold-change): pseudobulks vs tcga skcm",
    hover_name="gene_symbol",
    # hover_data=["log2foldchange_real", "log2foldchange_pseudo"],
)
# fig.update_xaxes(range=(-25, 25))
# fig.update_yaxes(range=(-25, 25))
fig.update_traces(marker=dict(size=3))
fig.show(renderer="png", scale=1, width=500, height=500)
fig.show(renderer="browser")

In [ ]:
fig.show(renderer="browser")

#### what's the overlap for significant genes between real, pseudo?

In [ ]:
df_gene_stats_by_immune["percentile_neglog10pval_pseudo"] = df_gene_stats_by_immune[
    "neglog10pval_pseudo"
].rank(pct=True)
df_gene_stats_by_immune["percentile_neglog10pval_real"] = df_gene_stats_by_immune[
    "neglog10pval_real"
].rank(pct=True)

In [ ]:
THRESHOLD = 0.9

df_gene_stats_by_immune["top_pseudo"] = (
    df_gene_stats_by_immune["percentile_neglog10pval_pseudo"] > THRESHOLD
)
df_gene_stats_by_immune["top_real"] = (
    df_gene_stats_by_immune["percentile_neglog10pval_real"] > THRESHOLD
)
df_gene_stats_by_immune["top_both"] = (
    df_gene_stats_by_immune["top_pseudo"] & df_gene_stats_by_immune["top_real"]
)
pd.crosstab(df_gene_stats_by_immune["top_pseudo"], df_gene_stats_by_immune["top_real"])

In [ ]:
# TODO - a good summary stat here would be the odds ratio of this contingency table

In [ ]:
# TODO - benjamini hochberg for a FDR-corrected significance test (use FDR of 0.1)

In [ ]:
# TODO - also do scatter for fold change

In [ ]:
# is it the same genes that showed up in the PCA analysis?